In [0]:
try:
  #configuration parameters
  server_name = "jdbc:sqlserver://azuresqllab.database.windows.net"
  database_name = "mssqldb "
  url = server_name + ";" + "databaseName=" + database_name + ";"
  username = "msadmin"
  password = "msoft@2020"
  
  MountPath = "/mnt/ADFandADBHands-on"
  
  #Function to read files from ADLS
  def readFilesFromAdls(fileName="",mntPath="",inferschema = "true",header = "true",ipfilepath = ""):
    if len(mntPath) == 0:
      raise Exception("Please enter valid mount name")
    if len(fileName) == 0:
      raise Exception("Please enter valid file name")
    FileName = fileName.split('.')[0]
    fileFormat = fileName.split('.')[1]
    filePath = mntPath + "/" + ipfilepath + FileName + "." + fileFormat
    df = spark.read.format(fileFormat).option('inferschema',inferschema).option("header", header).load(filePath)
    return df
  
  #Function to do union all operation of all the data frames
  def unionall(dfs):
    if len(dfs) > 1:
      return dfs[0].union(unionall(dfs[1:]))
    else:
      return dfs[0]
  
  #Function to save files into ADLS 
  def writeFilesIntoAdls(df="",fileName="",mntPath="",ippath=""):
    source = mntPath + "/" + ippath + "/"
    tempFilePath = source + "temp/" + fileName
    filePath = source + fileName
    df.write.format("csv").save(path = tempFilePath,mode="overwrite")
    fn = list(filter(lambda x:x[1].endswith("csv"),dbutils.fs.ls(tempFilePath)))[0][1]
    dbutils.fs.mv(tempFilePath+"/"+fn,filePath)
    dbutils.fs.rm(source + "temp",True)
  
  #Function to write files into ASQL
  def copyToAsql(df,fileName = "",url = "",username = "",password = "",mode = "overwrite"):
    if len(url) == 0:
      raise Exception("Enter valid server and database credentials")
    if len(username) == 0 or len(password) == 0:
      raise Exception("Enter valid username and password credentials")
    if len(fileName) == 0:
      raise Exception("Enter valid file name")
    tableName = '[' + 'stg_' + fileName.split('.')[0] + ']'
    df.write.format("jdbc").mode(mode).option("url", url).option("dbtable", tableName).option("user", username).option("password", password).save()
    
  empFiles = ["employees1.parquet","employees2.parquet","employees3.parquet","employees4.parquet","employees5.parquet","employees6.parquet"]
  empDf = []
  for employee in empFiles:
    df = readFilesFromAdls(fileName = employee,mntPath = MountPath)
    empDf.append(df)
  emp = unionall(empDf).distinct()
  emp.show()
  FileName = "employee_parquet_" + dbutils.widgets.get("empno") +".csv"
  writeFilesIntoAdls(df=emp,fileName=FileName,mntPath=MountPath,ippath=dbutils.widgets.get("empno"))
  copyToAsql(df=emp,fileName = Filename,url = url,username = username,password = password)
except Exception as x:
  print(x)

+---+-------------+----------+-----------+------+
 ID| Name|Department| Manager|Salary|
+---+-------------+----------+-----------+------+
 1| Robin Hood| Foo| Sally| 200|
 3| Friar Tuck| Foo| Robin Hood| 100|
 2|Arsene Wenger| Bar| Friar Tuck| 50|
 6|Dimi Berbatov| Foo|Little John| 50|
 5|Sam Allardyce| Bar| Sally| 250|
 4| Little John| Foo| Robin Hood| 100|
+---+-------------+----------+-----------+------+

An error occurred while calling o225.getArgument.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named empno is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.getArgument(NotebookArguments.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
import sklearn